In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.iolib.summary2 import summary_col
import os

In [3]:
os.chdir("C:\\Users\\Administrator\\Documents\\personal\\mrw1992\\mrw1992")

In [4]:
data = pd.read_csv("data/mrw.csv")

#### Reformatting the data

In [5]:
#convenience stuff
data = data.drop(data.columns[0], axis = 1)
data = data.set_index('country')

#creating desired variables
data['lngd'] = np.log(data['popgrowth']/100 + 0.05)
data['ls'] = np.log(data['i_y']/100)
data['const'] = 1
data['ls_lngd'] = data['ls'] - data['lngd']

# Estimating the Equations

We estimate this equation first

\begin{align}
log(Y/L) = a + (\alpha/(1 - \alpha))log(s) - (\alpha/(1 - \alpha))log(n + g + \delta) + \epsilon
\end{align}

where:
a is the constant in the assumed $log(A)$ equation  
$\epsilon$ is the country shock for $log(a)$  
$\alpha$ is the share of capital  
s is the exogenous savings rate (measured with average share of investment in GDP)  
n is the exogenous population growth rate  
g is the exogenous tfp growth rate  
$\delta$ is the depriciation rate of capital  

This corresponds to equation 7 in the paper

In [7]:
#subsetting data
data_reg = data.loc[data['n'] == 1, :] #no oil
data_d = data_reg[data_reg.i == 1] #pop in 1960 less than 1 mil
data_oecd = data_reg[data_reg.o == 1] #oecd

Unrestricted Model

In [8]:
#no restrictions on coefficients here
reg1 = sm.OLS(endog = np.log(data_reg['rgdpw85']),
             exog = data_reg[['const', 'ls', 'lngd']],
             missing = 'drop').fit()

reg2 = sm.OLS(endog = np.log(data_d['rgdpw85']),
             exog = data_d[['const', 'ls', 'lngd']],
             missing = 'drop').fit()

reg3 = sm.OLS(endog = np.log(data_oecd['rgdpw85']),
             exog = data_oecd[['const', 'ls', 'lngd']],
             missing = 'drop').fit()

Restricted Model

In [9]:
#coeff(log(savings)) = -coeff(log(n+g+d))
regr1 = sm.OLS(endog = np.log(data_reg['rgdpw85']),
             exog = data_reg[['const', 'ls_lngd']],
             missing = 'drop').fit()

regr2 = sm.OLS(endog = np.log(data_d['rgdpw85']),
             exog = data_d[['const', 'ls_lngd']],
             missing = 'drop').fit()

regr3 = sm.OLS(endog = np.log(data_oecd['rgdpw85']),
             exog = data_oecd[['const', 'ls_lngd']],
             missing = 'drop').fit()

Making table 1

In [10]:
info_dict = {'R^2': lambda x: x.rsquared,
            'N': lambda x: x.nobs,
            's.e.e.': lambda x: np.sqrt(x.scale),
            'Implied α': lambda x: f"{x.params[1]/(1 + x.params[1]):.2f}"}

results_unres = summary_col(results = [reg1, reg2, reg3],
                           float_format='%0.2f',
                           stars = True,
                           model_names = ['Non-Oil',
                                         'Intermediate',
                                         'OECD'],
                           info_dict = info_dict,
                           regressor_order = ['const',
                                             'ls',
                                             'lngd'])


results_res = summary_col(results = [regr1, regr2, regr3],
                           float_format='%0.2f',
                           stars = True,
                           model_names = ['Non-Oil',
                                         'Intermediate',
                                         'OECD'],
                           info_dict = info_dict,
                           regressor_order = ['const',
                                             'ls',
                                             'lngd'])


results_res.add_title('Restricted Regressions')
results_unres.add_title('Unrestricted Regressions')
print(results_unres)
print('')
print(results_res)

       Unrestricted Regressions
          Non-Oil  Intermediate   OECD 
---------------------------------------
const     5.43***  5.35***      8.02***
          (1.58)   (1.54)       (2.52) 
ls        1.42***  1.32***      0.50   
          (0.14)   (0.17)       (0.43) 
lngd      -1.99*** -2.02***     -0.74  
          (0.56)   (0.53)       (0.85) 
R^2       0.6009   0.5989       0.1059 
N         98.0000  75.0000      22.0000
s.e.e.    0.6891   0.6106       0.3774 
Implied α 0.59     0.57         0.33   
Standard errors in parentheses.
* p<.1, ** p<.05, ***p<.01

        Restricted Regressions
          Non-Oil Intermediate   OECD 
--------------------------------------
const     6.87*** 7.09***      8.62***
          (0.12)  (0.15)       (0.53) 
ls_lngd   1.49*** 1.43***      0.55   
          (0.12)  (0.14)       (0.37) 
R^2       0.5974  0.5917       0.1031 
N         98.0000 75.0000      22.0000
s.e.e.    0.6885  0.6119       0.3684 
Implied α 0.60    0.59         0.36   
Standar